## Start

In [1]:
import json
import numpy as np
import os
import torch
import tqdm
from shutil import copyfile

# Detectron imports
from detectron2.engine import launch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.data import build_detection_test_loader, MetadataCatalog

# Project imports
import core.datasets.metadata as metadata

from core.setup import setup_config, setup_arg_parser
from offline_evaluation import compute_average_precision, compute_probabilistic_metrics, compute_calibration_errors
from probabilistic_inference.probabilistic_inference import build_predictor
from probabilistic_inference.inference_utils import instances_to_json

from train_utils import ActiveTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
##setup inference args, this also contains all the training args
arg_parser = setup_arg_parser()
args = arg_parser.parse_args("")
# Support single gpu inference only.
args.num_gpus = 1
args.dataset_dir = '/public-dataset/BDD/bdd100k'
args.test_dataset = 'bdd_val'
args.config_file = '/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml'
args.inference_config = '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml'
args.random_seed = 1000
args.resume=False
print("Command Line Args:", args)

Command Line Args: Namespace(config_file='/home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout.yaml', dataset_dir='/public-dataset/BDD/bdd100k', dist_url='tcp://127.0.0.1:50162', eval_only=False, inference_config='/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml', iou_correct=0.7, iou_min=0.1, machine_rank=0, min_allowed_score=0.0, num_gpus=1, num_machines=1, opts=[], random_seed=1000, resume=False, test_dataset='bdd_val')


In [3]:
# run this once per session only
cfg = setup_config(args, random_seed=args.random_seed, is_testing=False)

Loading config /home/richard.tanai/cvpr2/pod_compare/src/configs/BDD-Detection/retinanet/../../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
Config '/home/richard.tanai/cvpr2/pod_compare/src/configs/Inference/bayes_od_mc_dropout.yaml' has no VERSION. Assuming it to be compatible with latest v2.


[02/07 10:53:58 detectron2]: Rank of current process: 0. World size: 1
[02/07 10:53:59 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------------
sys.platform            linux
Python                  3.8.5 (default, Sep  4 2020, 07:30:14) [GCC 7.3.0]
numpy                   1.19.5
detectron2              0.3 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 10.2
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.7.1 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torch
PyTorch debug build     False
GPU available           True
GPU 0                   Tesla V100-SXM2-32GB (arch=7.0)
CUDA_HOME               /usr/local/cuda
Pillow                  8.1.0
torchvision             0.8.2 @/opt/anaconda3/envs/pod/lib/python3.8/site-packages/torchvision
torchvision arch flags  3.5, 5.0

[02/07 10:53:59 detectron2]: Full config saved to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_1000/config.yaml


In [4]:
#cfg = setup_config(args, random_seed=args.random_seed, is_testing=True)

# Make sure only 1 data point is processed at a time. This simulates
# deployment.
cfg.defrost()
cfg.DATALOADER.NUM_WORKERS = 32
#cfg.SOLVER.IMS_PER_BATCH = 1

cfg.MODEL.DEVICE = device.type

# Set up number of cpu threads
torch.set_num_threads(cfg.DATALOADER.NUM_WORKERS)

# Create inference output directory and copy inference config file to keep
# track of experimental settings
inference_output_dir = os.path.join(
    cfg['OUTPUT_DIR'],
    'inference',
    args.test_dataset,
    os.path.split(args.inference_config)[-1][:-5])
os.makedirs(inference_output_dir, exist_ok=True)
copyfile(args.inference_config, os.path.join(
    inference_output_dir, os.path.split(args.inference_config)[-1]))

# Get category mapping dictionary:
train_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
test_thing_dataset_id_to_contiguous_id = MetadataCatalog.get(
    args.test_dataset).thing_dataset_id_to_contiguous_id

# If both dicts are equal or if we are performing out of distribution
# detection, just flip the test dict.
if (train_thing_dataset_id_to_contiguous_id == test_thing_dataset_id_to_contiguous_id) or (
        cfg.DATASETS.TRAIN[0] == 'coco_not_in_voc_2017_train'):
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
else:
    # If not equal, two situations: 1) BDD to KITTI and 2) COCO to PASCAL
    cat_mapping_dict = dict(
        (v, k) for k, v in test_thing_dataset_id_to_contiguous_id.items())
    if 'voc' in args.test_dataset and 'coco' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.COCO_TO_VOC_CONTIGUOUS_ID.items())
    elif 'kitti' in args.test_dataset and 'bdd' in cfg.DATASETS.TRAIN[0]:
        dataset_mapping_dict = dict(
            (v, k) for k, v in metadata.BDD_TO_KITTI_CONTIGUOUS_ID.items())
    else:
        ValueError(
            'Cannot generate category mapping dictionary. Please check if training and inference datasets are compatible.')
    cat_mapping_dict = dict(
        (dataset_mapping_dict[k], v) for k, v in cat_mapping_dict.items())

# Build predictor
model = build_model(cfg)

DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            "/home/richard.tanai/cvpr2/pod_compare/data_backup/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/model_final.pth", resume=False)

#for debug purposes
cfg.ACTIVE_LEARNING.START_N = 100 

trainer = ActiveTrainer(cfg, model)
#trainer.resume_or_load(resume=True)

#predictor = build_predictor(cfg, model)



[02/07 10:54:03 fvcore.common.checkpoint]: Loading checkpoint from /home/richard.tanai/cvpr2/pod_compare/data_backup/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_0/model_final.pth
[02/07 10:54:07 d2.data.datasets.coco]: Loading /public-dataset/BDD/bdd100k/labels/train_coco_format.json takes 4.15 seconds.
[02/07 10:54:07 d2.data.datasets.coco]: Loaded 69863 images in COCO format from /public-dataset/BDD/bdd100k/labels/train_coco_format.json
[02/07 10:54:10 d2.data.build]: Removed 458 images with no usable annotations. 69405 images left.
[02/07 10:54:12 d2.data.build]: Distribution of instances among all 7 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|    car     | 713211       |    bus     | 11672        |   truck    | 29971        |
|   person   | 91349        |   rider    | 4517         |    bike    | 7210         |

In [ ]:
# 10000 is already started in the init
# epoch is 20, just an arbitrary number lol

predictor = build_predictor(cfg, model)

test_data_loader = build_detection_test_loader(
    cfg, dataset_name=args.test_dataset)
train_step = 1
label_per_step = 100
while(1):
    print(f"performing train step {train_step}")
    trainer.train()
    torch.save(model.state_dict(), f"outputs/checkpoint_step{train_step}.pth")
    if len(trainer.dataset) >= label_per_step:
        trainer.dataset.label_randomly(label_per_step)
    elif len(trainer.dataset) > 0:
        trainer.datasaet.label_randomly(len(trainer.dataset))
    else:
        break
    trainer.rebuild_trainer()
    train_step += 1

performing train step 1
[02/03 19:54:32 d2.engine.train_loop]: Starting training from iteration 0
[02/03 19:54:47 d2.utils.events]:  eta: 0:00:39  iter: 99  total_loss: 0.1649  loss_cls: 0.04713  loss_box_reg: 0.1227  time: 0.1348  data_time: 0.0030  lr: 0.00024975  max_mem: 2432M
[02/03 19:55:01 d2.utils.events]:  eta: 0:00:26  iter: 199  total_loss: 0.2076  loss_cls: 0.05903  loss_box_reg: 0.1383  time: 0.1342  data_time: 0.0029  lr: 0.0004995  max_mem: 2432M
[02/03 19:55:14 d2.utils.events]:  eta: 0:00:13  iter: 299  total_loss: 0.2091  loss_cls: 0.05158  loss_box_reg: 0.1496  time: 0.1338  data_time: 0.0029  lr: 0.00074925  max_mem: 2432M
[02/03 19:55:28 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_333/model_final.pth
[02/03 19:55:28 d2.utils.events]:  eta: 0:00:00  iter: 399  total_loss: 0.2143  loss_cls: 0.05284  loss_box_reg: 0.1314  time: 0.1339  data_time

[02/03 20:02:42 d2.utils.events]:  eta: 0:00:53  iter: 1199  total_loss: 0.1498  loss_cls: 0.02695  loss_box_reg: 0.1131  time: 0.1342  data_time: 0.0026  lr: 0.0025  max_mem: 2432M
[02/03 20:02:56 d2.utils.events]:  eta: 0:00:40  iter: 1299  total_loss: 0.158  loss_cls: 0.01868  loss_box_reg: 0.1294  time: 0.1342  data_time: 0.0028  lr: 0.0025  max_mem: 2432M
[02/03 20:03:09 d2.utils.events]:  eta: 0:00:27  iter: 1399  total_loss: 0.165  loss_cls: 0.02679  loss_box_reg: 0.1155  time: 0.1343  data_time: 0.0031  lr: 0.0025  max_mem: 2432M
[02/03 20:03:23 d2.utils.events]:  eta: 0:00:13  iter: 1499  total_loss: 0.1426  loss_cls: 0.01527  loss_box_reg: 0.1248  time: 0.1343  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:03:36 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_333/model_final.pth
[02/03 20:03:37 d2.utils.events]:  eta: 0:00:00  iter: 1599  total_lo

[02/03 20:11:17 d2.utils.events]:  eta: 0:02:14  iter: 1399  total_loss: 0.1254  loss_cls: 0.01125  loss_box_reg: 0.1139  time: 0.1344  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:11:30 d2.utils.events]:  eta: 0:02:01  iter: 1499  total_loss: 0.1187  loss_cls: 0.00807  loss_box_reg: 0.09897  time: 0.1344  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:11:44 d2.utils.events]:  eta: 0:01:47  iter: 1599  total_loss: 0.09336  loss_cls: 0.008918  loss_box_reg: 0.0817  time: 0.1343  data_time: 0.0031  lr: 0.0025  max_mem: 2432M
[02/03 20:11:57 d2.utils.events]:  eta: 0:01:34  iter: 1699  total_loss: 0.1094  loss_cls: 0.007791  loss_box_reg: 0.09254  time: 0.1342  data_time: 0.0028  lr: 0.0025  max_mem: 2432M
[02/03 20:12:11 d2.utils.events]:  eta: 0:01:20  iter: 1799  total_loss: 0.09628  loss_cls: 0.01202  loss_box_reg: 0.07901  time: 0.1343  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:12:24 d2.utils.events]:  eta: 0:01:07  iter: 1899  total_loss: 0.101  los

[02/03 20:19:51 d2.engine.hooks]: Overall training speed: 2798 iterations in 0:06:16 (0.1346 s / it)
[02/03 20:19:51 d2.engine.hooks]: Total training time: 0:06:17 (0:00:01 on hooks)
performing train step 8
[02/03 20:19:51 d2.engine.train_loop]: Starting training from iteration 0
[02/03 20:20:04 d2.utils.events]:  eta: 0:06:56  iter: 99  total_loss: 0.0966  loss_cls: 0.00805  loss_box_reg: 0.08493  time: 0.1342  data_time: 0.0030  lr: 0.0025  max_mem: 2432M
[02/03 20:20:18 d2.utils.events]:  eta: 0:06:42  iter: 199  total_loss: 0.08495  loss_cls: 0.003859  loss_box_reg: 0.07971  time: 0.1341  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:20:31 d2.utils.events]:  eta: 0:06:29  iter: 299  total_loss: 0.09162  loss_cls: 0.00396  loss_box_reg: 0.08664  time: 0.1340  data_time: 0.0029  lr: 0.0025  max_mem: 2432M
[02/03 20:20:45 d2.utils.events]:  eta: 0:06:16  iter: 399  total_loss: 0.06917  loss_cls: 0.003399  loss_box_reg: 0.06557  time: 0.1340  data_time: 0.0030  lr: 0.0025  ma

In [ ]:
len(trainer.dataset.pool)

In [ ]:
# coco has 80000 images
# the max iter is 90000 and batchsize of 4, making it about 4.5epochs
# for this study 4 epochs will be used instead
# no weights refresh will be used
# hooks are now rebuilt

In [4]:
## prediction outputs
# in what order are the outputs arranged, they are arranged by the max clas pred score


# to do
# change test dataloader to pool dataloader
# use 

In [ ]:
final_output_list = []
cls_score_list = []
box_score_list = []


if not args.eval_only:
    with torch.no_grad():
        with tqdm.tqdm(total=len(test_data_loader)) as pbar:
            for idx, input_im in enumerate(test_data_loader):
                #print(input_im.size)
                outputs = predictor(input_im)
                final_output_list.extend(
                    instances_to_json(
                        outputs,
                        input_im[0]['image_id'],
                        cat_mapping_dict))
                results = outputs
                
                cls_preds = results.pred_cls_probs.cpu().numpy()
                predicted_boxes = results.pred_boxes.tensor.cpu().numpy()
                predicted_covar_mats = results.pred_boxes_covariance.cpu().numpy()
                
                box_score = np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()
                #mean of the max confidence
                cls_score = cls_preds.max(axis=1).mean()
                #change cls_score to entropy next time
                box_score_list.append(box_score)
                cls_score_list.append(cls_score)
                
                pbar.update(1)
                
                
cls_score_rank = np.array(cls_score_list).argsort().argsort()
box_score_rank = (-np.array(box_score_list)).argsort().argsort()
total_sort = np.argsort(cls_score_rank + box_score_rank)
idx_to_label = total_sort[:10000].tolist()

In [ ]:
#sort from most uncertain to least uncertain

In [134]:
cls_score_rank = np.array(cls_score_list).argsort().argsort()
box_score_rank = (-np.array(box_score_list)).argsort().argsort()
total_sort = np.argsort(cls_score_rank + box_score_rank)
idx_to_label = total_sort[:3].tolist()

In [12]:
def compute_cls_entropy(cls_preds, merge="mean"):
    
    assert len(cls_preds.shape) == 2
    
    ent = np.array([])
    for det_preds in cls_preds:
        ent = np.append(ent, (-det_preds*np.log(det_preds)).sum())
    
    print(ent)
    if merge == "mean":
        
        return ent.mean()
    
    elif merge == "max":
        return ent.max()
    
    else:
        raise ValueError('Invalid detection merge mode for entropy {}.'.format(merge))

In [6]:
def compute_cls_max_conf(cls_preds, merge="mean"):
    assert len(cls_preds.shape) == 2
    
    if merge == "mean":
        
        return cls_preds.max(axis=1).mean()
    
    elif merge == "max":
        return cls_preds.max(axis=1).max()
    
    else:
        raise ValueError('Invalid detection merge mode for max_conf {}.'.format(merge))

In [7]:
#loop using entropy
train_step = 1
label_per_step = 100
# cfg.ACTIVE_LEARNING.OUT_DIR
out_dir = "outputs_v1_10k"

# entropy or max_conf

# cfg.ACTIVE_LEARNING.DET_CLS_SCORE
det_cls_score = "entropy"

#cfg.ACTIVE_LEARNING.DET_CLS_MERGE_MODE
det_cls_merge_mode = "mean"

# cls score and box score weighted sum factor, 1 is full cls_score
# cfg.ACTIVE_LEARNING.W_CLS_SCORE
w_cls_score = 1

os.makedirs(out_dir, exist_ok=True)


while(1):
    print(f"performing train step {train_step}")
    trainer.train()
    torch.save(model.state_dict(), f"{out_dir}/checkpoint_step{train_step}.pth")

    pool_loader = trainer.build_pool_dataloader()

    final_output_list = []
    cls_score_list = []
    box_score_list = []

    predictor = build_predictor(cfg, model)

    if not args.eval_only:
        with torch.no_grad():
            with tqdm.tqdm(total=len(pool_loader)) as pbar:
                for idx, input_im in enumerate(pool_loader):
                    #print(input_im.size)
                    outputs = predictor(input_im)
                    final_output_list.extend(
                        instances_to_json(
                            outputs,
                            input_im[0]['image_id'],
                            cat_mapping_dict))
                    results = outputs

                    cls_preds = results.pred_cls_probs.cpu().numpy()
                    predicted_boxes = results.pred_boxes.tensor.cpu().numpy()
                    predicted_covar_mats = results.pred_boxes_covariance.cpu().numpy()

                    box_score = np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()
                    #mean of the max confidence pre detection
                    if det_cls_score == "entropy":
                        cls_score = compute_cls_entropy(cls_preds, det_cls_merge_mode) #entropy, mean default
                    elif det_cls_score == "max_conf":
                        cls_score = compute_cls_max_conf(cls_preds, det_cls_merge_mode)
                    else:
                        raise ValueError('Invalid det_cls_score {}.'.format(det_cls_score))
                        
                    box_score_list.append(box_score)
                    cls_score_list.append(cls_score)

                    pbar.update(1)
                    if idx > label_per_step*2:
                        print(f"the length of the pool is {len(pool_loader)}")
                        break


    cls_score_rank = np.array(cls_score_list).argsort().argsort()
    box_score_rank = (-np.array(box_score_list)).argsort().argsort()

    #possible weighted fusion can be added here
    total_sort = np.argsort((w_cls_score)*cls_score_rank + (1-w_cls_score)*box_score_rank)
    

    if len(trainer.dataset.pool) >= label_per_step:
        idx_to_label = total_sort[:label_per_step].tolist()
        trainer.dataset.label(idx_to_label)
        break
    elif len(trainer.dataset.pool) > 0:
        trainer.dataset.label_randomly(len(trainer.dataset.pool))
    else:
        break
    trainer.rebuild_trainer()
    train_step += 1

performing train step 1
[02/07 10:54:12 d2.engine.train_loop]: Starting training from iteration 0
[02/07 10:54:36 fvcore.common.checkpoint]: Saving checkpoint to /home/richard.tanai/cvpr2/pod_compare/data/BDD-Detection/retinanet/retinanet_R_50_FPN_1x_reg_cls_var_dropout/random_seed_1000/model_final.pth
[02/07 10:54:36 d2.utils.events]:  eta: 0:00:00  iter: 49  total_loss: 0.4013  loss_cls: 0.1315  loss_box_reg: 0.2638  time: 0.4352  data_time: 0.0095  lr: 0.00012488  max_mem: 8404M
[02/07 10:54:36 d2.engine.hooks]: Overall training speed: 48 iterations in 0:00:20 (0.4352 s / it)
[02/07 10:54:36 d2.engine.hooks]: Total training time: 0:00:21 (0:00:00 on hooks)
[02/07 10:54:37 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]


  0%|          | 202/69305 [01:13<6:58:53,  2.75it/s]


the length of the pool is 69305


In [13]:
compute_cls_entropy(cls_preds)

[0.6735816  0.65458161 0.57290536 0.6810838  1.10173106 1.17634439
 1.14614856 0.79682165 0.89564419 0.97529751 1.5059154  1.09316909
 1.17639792 1.2976594  1.07864618 1.14987123 1.50417042 1.05135262
 1.09704983 1.04031229 0.98953509 0.98289454 1.2418803  0.83059418
 0.94581699 0.97859418 1.22000003 1.13844299 1.32917404 1.27367914
 1.0694921  1.26989162 0.89865428 0.91713995 0.8677687  1.13772607
 0.81497902 1.30278134 0.93699443 1.28821516 1.08093369 1.39140105
 1.48048925 0.87731302 1.29598498 0.68839628 0.88472193 1.38740277
 0.83700693 0.83375919 1.34075356 0.88505739 0.73032075 0.90696609
 1.0937072  0.97115868 0.7281034  0.66081011 0.99408495 0.6231001
 0.61153191 1.05822253 0.91620499 0.69298375 0.70925486 1.12173009
 1.09421694 0.92841303 0.87815624 0.79633689 0.91192293 0.75956959
 0.83807367 0.80594772 0.60729116 0.70997304 0.90303493 0.71641445
 0.62639457 0.77782333 0.61238778 0.64644831 0.67987573 1.02581084
 0.69630688 1.05626202 0.58497572 0.58348227 0.70365292 0.77111

0.922051522731781

In [8]:
cls_score_list

[0.6347876214064084,
 0.868465895652771,
 0.7172797167301178,
 0.6561238023638726,
 0.5725444634445012,
 0.7031967601180077,
 0.6380870440602302,
 0.6820839682350988,
 0.8562709707574746,
 0.641229946570224,
 0.6777238786220551,
 0.815227597951889,
 0.9003307051518384,
 0.6449304457570686,
 0.6527539189311041,
 0.6444879284460251,
 0.827795016169548,
 0.5849748653369945,
 0.7724103718996048,
 0.7825853398442268,
 0.7859344157305631,
 0.9280733913183212,
 0.7855429524725134,
 0.9066429895162582,
 0.7241702654957771,
 0.7221918654441833,
 0.9411766767501831,
 0.7827674907942613,
 0.81192287504673,
 0.6476578989624977,
 0.8705913777254066,
 0.7119450056552887,
 0.9234286105632782,
 0.8602607727050782,
 0.7171079965605252,
 0.8297125205397606,
 0.6850364674863062,
 0.7304124458767902,
 0.8570292297005654,
 0.7242111459374427,
 0.7089610090851783,
 0.7781022521853447,
 0.8282436151057482,
 0.8120404353737831,
 1.026125004887581,
 0.6144294988135902,
 0.7408296513557434,
 0.7444659778475762,

In [19]:
b = np.array([[1,2,3],[1,2,3]])
b.mean().max(axis=0)

2.0

In [9]:
cls_score_rank

array([ 24, 169,  89,  41,   3,  73,  26,  58, 165,  29,  55, 145, 180,
        34,  38,  32, 149,   5, 123, 127, 131, 189, 130, 183,  97,  92,
       191, 128, 142,  37, 170,  80, 186, 167,  87, 151,  61, 103, 166,
        98,  77, 126, 150, 143, 201,  13, 108, 110,  28,  48,  66, 111,
        78, 168, 140,  96,  85,  16,  19, 117, 193, 129, 162, 164,  47,
        42,   1, 159, 171, 109,  60,  67,  68, 179,  27, 107, 188, 163,
        69,  63,  64, 106,  79, 120,  25,  57,  21,  59,   7, 158,  18,
        40,  54, 144, 200, 102, 141,  95,  43,  22,  49, 113,  52,  71,
       114,  84,  53,  11,  35, 104,  12,  15, 136, 105,  76,  33, 192,
        31, 135, 182,  51, 195,   9,  20, 132, 199, 122,  91,  62,   2,
       198, 178, 172,  88,  50,   8,  39, 138, 153, 196,  44, 139,  46,
        36, 152, 181, 190, 124, 160,  74, 156,   0,  83,  45,  70, 184,
        30, 125, 146, 133,  99, 157,  90,  82, 100,  81, 197, 115, 119,
         4, 161, 173, 137, 118, 101,  56, 147,  94, 194, 187, 11

In [9]:
(w_cls_score)*cls_score_rank + (1-w_cls_score)*box_score_rank

array([105,  68,  39,  59,   6,  87,  33,  53, 168,  56,   5, 179, 145,
        95,  54,  19, 146,  52, 159,  40,  82, 177, 101,  80,  45, 141,
       171, 199,  77,  10, 129, 139, 152, 161, 181, 111,  51, 138, 142,
        96,  48, 114, 133, 121, 119,  61, 167, 110,   3,  44, 109,  12,
        57, 150, 156, 112, 127, 106,  16, 176, 157,  60,  49,  65,  73,
        22,  30, 174, 184, 166,  47, 140, 195,  89,  18, 125, 189, 128,
        74,  24, 194,  76, 148, 130, 134,  62,   8,  32,   0,  78,  23,
       163,  34,  14, 196,  98, 123,  25,  71,  31, 104,  93, 155, 175,
        94,  63,  50,  66,  20,  79,   7,  58,   9, 115,  46,  36, 186,
        91,  38, 178,  90, 185,  67, 107,  75, 188, 201, 172,  85,   1,
       149, 117, 126, 192, 160,  81,  43, 124, 198, 103,  35,  13, 113,
        26,  55, 122, 180, 197, 143,  84,  83,   2, 191,  15, 118, 187,
        92, 100, 183, 108, 151, 136, 165,  64,  97, 169, 131, 158, 147,
         4,  70, 154, 153,  99,  37, 173, 164,  86, 190, 162, 20

In [40]:
def mult_diag(x):
    print(x)
    return x.diagonal().prod()

In [74]:
np.array([mat.diagonal().prod() for mat in predicted_covar_mats]).mean()

1481842400000.0

In [ ]:
np.apply_over_axes